Calculo de percentual de finalizados em relação ao total de inadimplentes abertos

In [ ]:
%pip install google-cloud-bigquery pymsteams
from google.cloud import bigquery
from collections import defaultdict
import pymsteams

webhooks = {
    "Bruno Pinho": f"https://prod-22.brazilsouth.logic.azure.com:443/workflows/8e2ee55840b641718825e81fda82edae/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=2xSvgPYaBU-_urpAAr_nNc2xOKaMiVXVS-qIo9bU2nw",
    "Francisco Miranda": f"https://prod-12.brazilsouth.logic.azure.com:443/workflows/3cc9fa6489384429a5c47c7948dac23f/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=LYfZGc_ZyeJkngOce6AuvEGhxyGtGlijUekb1RSZZx4",
    "João Pradella": f"https://prod-21.brazilsouth.logic.azure.com:443/workflows/54a32eb759a5438c9b94afa86ae3b11b/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=4ebMmVvgrIzQxlr7sFIccPAtL9-w6m47wxIEG6FYn60",
    "Rogério Ferreira Silva": f"https://prod-10.brazilsouth.logic.azure.com:443/workflows/49d62273f0354e8e844d3ac81a43e7d6/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=7sPDGCXeSQp84gxuyjz3dOBg-ZZJ59mwKX1C0oGKA60"
}


client = bigquery.Client()

def buscar_dados_bq():
    """Consulta dados de serviços no BigQuery"""
    query = """
        SELECT
          filial,
          CASE
            WHEN descricaoEvento = 'Aberto' THEN 'Aberto'
            WHEN descricaoEvento IN ('[Finalizado] Moto paga', '[Finalizado] Moto recolhida', '[Cancelado] Pagamento realizado pelo cliente') THEN 'Finalizado'
            ELSE 'Outro'
          END AS identificacaoEvento
        FROM `dm-mottu-aluguel.flt_servicos_rua.eventos_rua`
        WHERE descricaoServico = 'Inadimplência'
          AND (
            (descricaoEvento = 'Aberto' AND nome_analista = 'Mottu Monitor')
            OR
            descricaoEvento IN ('[Finalizado] Moto paga', '[Finalizado] Moto recolhida')
          )
          AND dataEvento >= TIMESTAMP_SUB(TIMESTAMP_TRUNC(CURRENT_TIMESTAMP(), DAY), INTERVAL 1 DAY)
          AND dataEvento < TIMESTAMP_TRUNC(CURRENT_TIMESTAMP(), DAY)
    """
    return client.query(query).result()

def processar_dados(rows):
    """Processa os dados do BigQuery e conta eventos por filial"""
    contagem = defaultdict(lambda: defaultdict(int))
    for row in rows:
        filial = row['filial']
        evento = row['identificacaoEvento']
        contagem[filial][evento] += 1
    return contagem

regionais = {
        "Rogério Ferreira Silva": {
            "Mottu Anápolis", "Mottu Arapiraca", "Mottu Brasília", "Mottu Camaçari", "Mottu Florianópolis",
            "Mottu Franca", "Mottu Imperatriz", "Mottu Joinville", "Mottu Juazeiro do Norte", "Mottu Londrina",
            "Mottu Maceió", "Mottu Marabá", "Mottu Maringá", "Mottu Montes Claros", "Mottu Mossoró", "Mottu Palmas",
            "Mottu Piracicaba", "Mottu Santarém", "Mottu São José do Rio Preto", "Mottu Sobral", "Mottu Uberaba",
            "Mottu Vitória da Conquista"
        }
        ,
        "Bruno Pinho": {
            "Mottu Belo Horizonte", "Mottu Boa Vista", "Mottu Campina Grande", "Mottu Campo Grande",
            "Mottu Caruaru", "Mottu Contagem", "Mottu Feira de Santana", "Mottu Juazeiro", "Mottu Macapá",
            "Mottu Parauapebas", "Mottu Porto Alegre", "Mottu Porto Velho", "Mottu Rio Branco",
            "Mottu São José dos Campos", "Mottu Sorocaba", "Mottu Uberlândia", "Mottu Vila Velha", "Mottu Vitória"
        }
        ,
       "Francisco Miranda": {
            "Mottu Ananindeua", "Mottu Aracaju", "Mottu Belém", "Mottu Cuiabá", "Mottu Curitiba", "Mottu Fátima",
            "Mottu Fortaleza", "Mottu Goiânia", "Mottu João Pessoa", "Mottu Manaus", "Mottu Maracanau", "Mottu Natal",
            "Mottu Olinda", "Mottu Parnamirim", "Mottu Piçarreia", "Mottu Recife", "Mottu Ribeirão Preto", "Mottu Salvador",
            "Mottu São Luís", "Mottu Teresina"
        }
        ,
       "João Pradella": {
            "Mottu Alagoinhas", "Mottu Aparecida de Goiânia", "Mottu Araçatuba", "Mottu Barreiras", "Mottu Bauru",
            "Mottu Bragança", "Mottu Caxias do Sul", "Mottu Criciúma", "Mottu Divinópolis", "Mottu Governador Valadares",
            "Mottu Ipatinga", "Mottu Itabuna", "Mottu Itajaí", "Mottu Jacarepaguá / RJ", "Mottu Juiz De Fora",
            "Mottu Linhares", "Mottu Niterói", "Mottu Parnaíba", "Mottu Pelotas", "Mottu Presidente Prudente",
            "Mottu Rio Verde", "Mottu Rondonópolis", "Mottu São Carlos", "Mottu Vila Isabel / RJ", "Mottu Vitória de Santo Antão"
        }
    }


def classificar_filial(filial):
    """Classifica a filial de acordo com o gestor regional"""
    for gestor, filiais in regionais.items():
        if filial in filiais:
            return gestor
    return "Não classificado"

def enviar_relatorio_teams(gestor, filiais_data):
    """Envia relatório consolidado para o gestor no Teams"""
    if gestor not in webhooks:
        print(f"Webhook não encontrado para {gestor}")
        return

    mensagem = pymsteams.connectorcard(webhooks[gestor])

    conteudo = "🚨 RESULTADO DE ONTEM - FILIAIS QUE MENOS PRODUZIRAM 🚨<br>"
    conteudo += "Inadimplências Abertas vs. Finalizadas<br><br>"

    for data in sorted(filiais_data, key=lambda x: x['percentual'], reverse=True):
        conteudo += (
            f"<b>{data['filial']}</b><br>"
            f"<b>Percentual:</b> {data['percentual']:.2f}%<br>"
            f"<b>Total de casos inadimplentes abertos:</b> {data['total']} | ✅ <b>Finalizados:</b> {data['finalizados']}<br><br>"
        )

    mensagem.text(conteudo)
    try:
        mensagem.send()
        print(f"Relatório enviado para {gestor}")
    except Exception as e:
        print(f"Erro ao enviar para {gestor}: {str(e)}")

def main():

    dados = buscar_dados_bq()
    contagem = processar_dados(dados)

    resultado_filtrado = []
    for filial, eventos in contagem.items():
        total = sum(eventos.values())
        finalizados = eventos.get('Finalizado', 0)
        percentual = (finalizados / total) * 100 if total > 0 else 0

        if percentual < 15:
            resultado_filtrado.append({
                'filial': filial,
                'percentual': percentual,
                'total': total,
                'finalizados': finalizados,
                'gestor': classificar_filial(filial)
            })

    relatorios = defaultdict(list)
    for item in resultado_filtrado:
        relatorios[item['gestor']].append(item)

    print("Enviando relatórios...")
    for gestor, dados in relatorios.items():
        enviar_relatorio_teams(gestor, dados)

    print("Processo concluído!")

if __name__ == "__main__":
    main()